In [ ]:
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe_group
from probeinterface.io import write_probeinterface
from probeinterface import generate_tetrode
from Tetrodes import create_and_print_probe, write_probe

Los probes se configuran como un arreglo de 4 tetrodos, donde es necesario especificar los canales asociados a cada electrodo del tetrodo y el numero del tetrodo. Esta información la tienes en tus excel, pero debes rotularla como columnas.

evitar utilizar multishanks, presentarlo como single electrode y presentar una columna de grupos en el excel.

Dev: canales del intan (0-63)  
Probe: numero de tetrodo (0-15)  
Chan: canal Matlab (1-64)  
TTR: numero de tetrodo Matlab (1-16)  

los pasos del pseudocodigo son los siguientes:  
1. Definicion de parametros  
2. Leer el archivo de Excel  
3. Extraer la columna 'Dev' para usarla en set_global_device_channel_indices  
4. Crear el ProbeGroup vacio  
5. Generar 16 tetrodos en 4 grupos de 2x2 
6. Asignar los índices de canales utilizando los datos de la columna 'Dev' y guardar   
7. Convertir a DataFrame  
8. Visualizar el grupo de tetrodos  

In [ ]:
# Leer el archivo de Excel
probe_name = 'TM24'
file_path = r'F:\sort\ariel\TM24\informacion_probe.xlsx'  # Reemplaza con la ruta a tu archivo Excel
df_excel = pd.read_excel(file_path)

# Extraer la columna 'Dev' para usarla en set_global_device_channel_indices
device_channel_indices = df_excel['Dev'].values

# Crear el ProbeGroup y los tetrodos como antes
probegroup = ProbeGroup()

# Definir las separaciones entre tetrodos y entre grupos
tetrode_spacing = 50
group_spacing = 1000

# Generar 16 tetrodos en 4 grupos de 2x2
for group_i in range(2):
    for group_j in range(2):
        base_x = group_i * group_spacing
        base_y = group_j * group_spacing
        
        for i in range(2):
            for j in range(2):
                tetrode = generate_tetrode()
                tetrode.move([base_x + i * tetrode_spacing, base_y + j * tetrode_spacing])
                probegroup.add_probe(tetrode)

# Asignar los índices de canales utilizando los datos de la columna 'Dev'
probegroup.set_global_device_channel_indices(device_channel_indices)
write_probe(probe_name, probegroup)

# Convertir a DataFrame
df = probegroup.to_dataframe()
print(df)

# Visualizar el grupo de tetrodos
plot_probe_group(probegroup, with_contact_id=True, with_device_index=True, same_axes=True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from probeinterface import Probe, ProbeGroup
from probeinterface import generate_linear_probe, generate_multi_shank
from probeinterface import combine_probes
from probeinterface.plotting import plot_probe

In [ ]:
from pathlib import Path

probe_name = 'probes/Anillo'
file_path = Path('probes/anillo_probe.xlsx')  # Reemplaza con la ruta a tu archivo Excel
df_excel = pd.read_excel(file_path)
device_channel_indices = df_excel['Dev'].values
contacts_id = df_excel['info'].values

multi_shank = generate_multi_shank(num_shank=4, num_columns=4, num_contact_per_column=1)
multi_shank.set_device_channel_indices(device_channel_indices)
multi_shank.set_contact_ids(device_channel_indices)

write_probe(probe_name, multi_shank)

In [ ]:

plot_probe(multi_shank, with_contact_id = True,
          with_device_index = True,
          show_channel_on_click = True)


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from probeinterface import Probe
from Tetrodes import create_and_print_probe, write_probe

# Ruta al archivo Excel
file_path = Path(r'C:\Users\Labcn\OneDrive - Universidad Católica de Chile\sorting_anillo\probes\anillo_probe.xlsx')

# Leer el archivo Excel
df_excel = pd.read_excel(file_path)
device_channel_indices = df_excel['Dev'].values
contacts_id = df_excel['info'].values

def create_16_channel_probe(device_channel_indices, contacts_id):
    # Crear una instancia de la sonda
    probe = Probe(ndim=2)
    
    # Definir los parámetros de la sonda
    num_channels = len(device_channel_indices)
    num_groups = 4
    channels_per_group = num_channels // num_groups
    vertical_spacing = 100  # 100 µm entre cada grupo de canales
    
    # Crear la matriz de posiciones de los contactos
    x_positions = []
    y_positions = []
    
    for group_idx in range(num_groups):
        for ch_idx in range(channels_per_group):
            x_positions.append(group_idx * 50)  # 50 µm de separación horizontal entre grupos
            y_positions.append(group_idx * vertical_spacing + ch_idx * 20)  # 20 µm de separación vertical entre canales dentro de un grupo
    
    contact_positions = np.column_stack((x_positions, y_positions))
    
    # Asignar los contactos a la sonda
    probe.set_contacts(positions=contact_positions, shapes='circle', shape_params={'radius': 10})
    
    # Asignar los índices de los canales y los IDs de los contactos
    probe.set_device_channel_indices(device_channel_indices)
    # probe.set_contact_ids(contacts_id)
    
    # Imprimir la información de la sonda
    print(probe)
    
    return probe

# Generar la sonda con los índices de canales del dispositivo y los IDs de los contactos
probe = create_16_channel_probe(device_channel_indices, contacts_id)

# Si quieres guardar el probe en un archivo, puedes usar el siguiente código:
# from probeinterface.io import write_probeinterface
# write_probeinterface(probe, '16_channel_probe.prb')

write_probe('test', probe)

# electrodo en version kilosort GUI

In [1]:
import kilosort

ModuleNotFoundError: No module named 'kilosort'

In [2]:
!pip install kilosort

In [ ]:
import numpy as np
import scipy.io

# Generar la disposición de los canales
n_groups = 4
n_channels_per_group = 4
n_total_channels = n_groups * n_channels_per_group

# Separación de canales dentro de un grupo y entre grupos
channel_spacing = 20  # Separación de 20um entre canales dentro de un grupo
group_spacing = 50    # Separación de 50um entre grupos

# Inicializar arrays para las coordenadas
xc = []
yc = []

# Generar las coordenadas
for group_idx in range(n_groups):
    # La posición en y aumenta para cada grupo, separados por group_spacing
    y_base = group_idx * group_spacing
    for channel_idx in range(n_channels_per_group):
        # Los canales dentro de cada grupo están alineados en x, separados por channel_spacing
        xc.append(channel_idx * channel_spacing)
        yc.append(y_base)

# Convertir a arrays de numpy
xc = np.array(xc)
yc = np.array(yc)

# Índices de los canales
chanMap = np.arange(1, n_total_channels + 1)

# Todos los canales pertenecen al mismo shank (puedes ajustarlo si es necesario)
kcoords = np.zeros(n_total_channels)

# Número total de canales
n_chan = n_total_channels

# Crear un diccionario con los datos de la probe
probe_data = {
    'chanMap': chanMap,
    'xc': xc,
    'yc': yc,
    'kcoords': kcoords,
    'n_chan': n_chan
}

from kilosort.io import save_probe

save_probe(probe, 'probes/probe_kilosort.json')

print("Archivo kilosort_probe_16ch.mat generado con éxito.")
